# Assignment 2: Monte Carlo (MC) Methods in Finance

In [ ]:
import numpy as np
from scipy.stats import norm

## Part I Option Valuation

## Part II Estimation of Sensitivities in MC

### Q1: Use bump-and-revalue method to estimate delta

## Part III Variance Reduction

### Q1: Compare the Asian call option result between the analytical expression and Monte-Carlo simulations

In [ ]:
from scipy.stats import norm

S0 = 100       # Initial stock price
K = 100        # Strike price
T = 1          # Time to maturity in years
r = 0.05       # Risk-free interest rate
sigma = 0.2    # Volatility
N = 252        # Number of steps/times we observe the stock price

# Adjust volatility and interest rates based on the characteristics of Asian options
sigma_hat = sigma * np.sqrt((2 * N + 1) / (6 * (N + 1)))
r_hat = (r - 0.5 * sigma**2 + sigma_hat**2) / 2

# Calculate d1 and d2
d1 = (np.log(S0 / K) + (r_hat + 0.5 * sigma_hat**2) * T) / (sigma_hat * np.sqrt(T))
d2 = d1 - sigma_hat * np.sqrt(T)

# Calculate Asian call option prices based on the BS model
A_call_Analytical_result = S0 * np.exp((r_hat - r) * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

print(A_call_Analytical_result)


In [ ]:
# Use Monte-Carlo simulation to compute Asian call option

# Given parameters
S0 = 100       # Initial stock price
K = 100        # Strike price
T = 1          # Time to maturity in years
r = 0.05       # Risk-free interest rate
sigma = 0.2    # Volatility
n_simulations = 100000  # Number of Monte Carlo simulations
n_steps = 252           # Number of steps/times we observe the stock price

# Function to simulate one geometric Brownian motion path
def simulate_gbm(S0, r, sigma, T, n_steps):
    dt = T/n_steps  # Time step
    # Simulate the increments of a standard Brownian motion
    dW = np.random.normal(scale=np.sqrt(dt), size=n_steps)
    # Calculate the log of stock prices using the GBM formula
    log_S = np.log(S0) + np.cumsum((r - 0.5 * sigma**2) * dt + sigma * dW)
    S = np.exp(log_S)  # Convert log prices back to normal prices
    return S

# Function to calculate the payoff of an Asian call option
def asian_call_payoff(S_path, K, r, T):
    geometric_mean = np.exp(np.mean(np.log(S_path)))  # Geometric mean of stock prices
    payoff = np.maximum(geometric_mean - K, 0)        # Call option payoff
    discounted_payoff = np.exp(-r * T) * payoff       # Discounted payoff
    return discounted_payoff

# Running the Monte Carlo simulation
np.random.seed()  # Set seed for reproducibility
payoffs = np.zeros(n_simulations)

for i in range(n_simulations):
    S_path = simulate_gbm(S0, r, sigma, T, n_steps)
    payoffs[i] = asian_call_payoff(S_path, K, r, T)

# Calculate the average payoff and hence the price of the option
A_call_Monte_Carlo = np.mean(payoffs)

print(f"A_call_Monte_Carlo = {A_call_Monte_Carlo}")
print(f"A_call_Analytical_result = {A_call_Analytical_result}")